In [3]:
import os
import pandas as pd

In [4]:
# data_path = 'C:/Users/samue/Downloads/ALL_DATA_NEW/ALL_DATA_NEW'
data_path = '/Users/sam/Downloads/ALL_DATA_NEW'

colnames = ['round_number', 'block_number', 'sequence_number', 'block_type', 'winning_probability',
'aaron_mood', 'probability_aaron_show', 'reward_value', 'reaction_time', 'choice',
'status_of_play', 'outcome', 'accumulated_outcomes', 'age', 'gender', 'major', 
'MCQ_Q1', 'MCQ_Q2', 'MCQ_Q3', 'MCQ_Q4', 'MCQ_Q5', 'MCQ_Q6', 'pre_game_strategy', 'wager', 'post_game_quiz_q1', 
'post_game_quiz_q2', 'post_game_quiz_q3', 'post_game_quiz_q3_other', 'post_game_quiz_correct',
'odds_guess_one_reply', 'odds_guess_one_did_win', 'odds_guess_two_reply', 'odds_guess_two_did_win',
'odds_guess_three_reply', 'odds_guess_three_did_win', 'id', 'treatment', 'email', 'craver']

data_full = pd.DataFrame(columns = colnames)

ID = 1
missed = []
trials = []

for exp in ['C', 'T']:
    exp_path = os.path.join(data_path, exp)

    for sub in [f for f in os.listdir(exp_path) if 'xls' in f]:
        excel_df = os.path.join(exp_path, sub)
        
        temp = pd.read_excel(excel_df, header = 0)
        
        if exp == 'C':
            t = 'control'
        else:
            t = 'test'
        
        if temp.shape[1] == 35 and 'T' not in sub and sum(temp['choice'] == 2) < 7 and temp.shape[0] > 599:

            # if int(sub.split(".xls")[0][-3:]) > 160:
            #     temp['batch'] = 1
            # else:
            #     temp['batch'] = 0

            temp['id'] = ID
            temp['treatment'] = t
            temp['email'] = sub
            
            if temp.loc[0, 'wager'] != 0:
                temp['craver'] = 0
            else:
                temp['craver'] = 1
            
            data_full = pd.concat([data_full, temp])
            ID += 1

In [5]:
data_full.to_csv('/Users/sam/Downloads/ALL_DATA_NEW/data_pilot.csv', index=False)

In [5]:
data_full.to_csv('C:/Users/samue/Downloads/ALL_DATA_NEW/data_pilot.csv', index=False)